In [1]:
#   SODAR     LON 103.77  LAT 19.28,                      LON 103.77  LAT 19.28,
# uv[0,20,56] =  -103.767  ,  19.3712       # uv[0,20,57] =  -103.681  ,  19.2907
# uv[0,20,56] =  -103.767  ,  19.2903*      # uv[0,20,56] =  -103.767  ,  19.2903*
# uv[0,20,55] =  -103.853  ,  19.2899*      # uv[0,18,56] =  -103.766  ,  19.1286 
# uv[0,19,56] =  -103.767  ,  19.2095*      # uv[0,20,55] =  -103.853  ,  19.2899* 
# uv[0,19,55] =  -103.852  ,  19.209 *      # uv[0,20,54] =  -103.938  ,  19.2894
# ds083 GFS NAM
# p1s1 p2s2 p7s7

In [1]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wrf 
from wrf import getvar, vinterp, ALL_TIMES, destagger, to_np, interplevel

C:\Users\dell\Anaconda3\lib\site-packages\xarray\core\merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
## f = '/mnt/sda2/exp6_llj/d03/20120816_d03_nam32_p1r1.nc'
f = '/../../../d03/20120816_d03_nam32_p1r1.nc'
ncfile = Dataset(f)
wrftime = getvar(ncfile, "times", timeidx=ALL_TIMES)
utc2loc = np.timedelta64(6,'h')
wrftime10m = pd.to_datetime(wrftime.values)-utc2loc
wrftimeh = pd.to_datetime(wrftime[::3].values)-utc2loc
timelabel = wrftimeh
date1, date2 = wrftimeh[0], wrftimeh[wrftimeh.size-1]
print(wrftimeh.size, date1, date2)

145 2012-08-16 12:00:00 2012-08-19 12:00:00


C:\Users\dell\Anaconda3\lib\site-packages\xarray\core\dataarray.py:217: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  elif isinstance(data, pd.Panel):


In [3]:
tv0 = getvar(ncfile, "tv", units="degC", timeidx=0)
tv0[2,24,20]


<xarray.DataArray 'tv' ()>
array(33.12079, dtype=float32)
Coordinates:
    XLONG    float32 -103.77844
    XLAT     float32 19.280983
    XTIME    float32 0.0
    Time     datetime64[ns] 2012-08-16T18:00:00
Attributes:
    FieldType:      104
    MemoryOrder:    XYZ
    description:    virtual temperature
    units:          degC
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LambertConformal(stand_lon=-101.25, moad_cen_lat=21.50001...
    _FillValue:     1e+20
    missing_value:  1e+20

In [5]:
#uv['Time'] = uv['Time'] - utc2loc
##  /mnt/sda2/exp6_llj/d03/

expruns = ['20120816_d03_gfs05_p1r1', '20120816_d03_gfs05_p1r3','20120816_d03_gfs05_p1r5',
           '20120816_d03_gfs05_p2r1', '20120816_d03_gfs05_p2r3','20120816_d03_gfs05_p2r5',
           '20120816_d03_nam32_p1r1', '20120816_d03_nam32_p1r3','20120816_d03_nam32_p1r5',
           '20120816_d03_nam32_p2r1', '20120816_d03_nam32_p2r3','20120816_d03_nam32_p2r5']
expnames = ['g1r1', 'g1r3', 'g1r5', 'g2r1', 'g2r3', 'g2r5', 
            'n1r1', 'n1r3', 'n1r5', 'n2r1', 'n2r3', 'n2r5']
print(expruns[0], expnames[0])

20120816_d03_gfs05_p1r1 g1r1


######   WRF TIME AVERAGE 30min at SODAR  #######
# (UV DIR, times, level, lat, lon) [0,:,:,20,56]
# SODAR   LAT 19.28, LON 103.77
k=0
lat, lon = 24, 20
int_levs = [.030, .040, .050, .100, .150, .200, .250, .300, .350, .400, .450, .500]
lats, lons = [lat-1, lat, lat+1], [lon-1, lon, lon+1]
sn, we = 'south_north', 'west_east'

for exprun in expruns:
    #exprun = '20120816_ds083_p1s1'
    #f= '/mnt/sda2/exp6_llj/d03/'+exprun+'.nc'
    f = '/../../../d03/'+exprun+'.nc'
    ncfile = Dataset(f)

    uv, wdir = getvar(ncfile, "uvmet_wspd_wdir", units="m s-1", timeidx=ALL_TIMES)
    tmpv = getvar(ncfile, "tv", units="degC", timeidx=ALL_TIMES)

    wind_ds = vinterp(ncfile, field=uv, vert_coord='ght_agl', interp_levels=int_levs, timeidx=ALL_TIMES, field_type='z')
    seriesw = wind_ds[:,:,lats,lons].mean(dim=[sn,we]).resample(Time='30min').mean(dim='Time')
    seriesw['Time'] = seriesw['Time'] - utc2loc

    wdir_ds = vinterp(ncfile, field=wdir, vert_coord='ght_agl', interp_levels=int_levs, timeidx=ALL_TIMES) 
    seriesd = wdir_ds[:,:,lats,lons].mean(dim=[sn,we]).resample(Time='30min').mean(dim='Time')
    seriesd['Time'] = seriesd['Time'] - utc2loc

    tmpv_ds = vinterp(ncfile, field=tmpv, vert_coord='ght_agl', interp_levels=int_levs, timeidx=ALL_TIMES) 
    seriest = tmpv_ds[:,:,lats,lons].mean(dim=[sn,we]).resample(Time='30min').mean(dim='Time')
    seriest['Time'] = seriest['Time'] - utc2loc

    ds1 = seriesw.to_dataset(name='uv30')
    ds2 = seriesd.to_dataset(name='wd30')
    ds3 = seriest.to_dataset(name='tv30')

    ds3.combine_first(ds2.combine_first(ds1)).to_netcdf('interp30/sodar_'+str(expnames[k])+'.nc')
    
    print('done '+str(k)+' '+exprun)
    k=k+1


In [6]:
######   WRF TIME AVERAGE 30m   #######
utc2loc = np.timedelta64(6,'h')
k=0
ztop=12
for exprun in expruns:
    #f= '../../../d03/'+exprun+'.nc'
    f = '/../../../d03/'+exprun+'.nc'
    ncfile = Dataset(f)

    ua = getvar(ncfile, "ua", units="ms-1", timeidx=ALL_TIMES)[:,:ztop,:,:]
    va = getvar(ncfile, "va", units="ms-1", timeidx=ALL_TIMES)[:,:ztop,:,:]
    wa = getvar(ncfile, "wa", units="ms-1", timeidx=ALL_TIMES)[:,:ztop,:,:]
    tv = getvar(ncfile, "tv", units="degC", timeidx=ALL_TIMES)[:,:ztop,:,:]
    z = getvar(ncfile, "z", units="m", timeidx=ALL_TIMES)[:,:ztop,:,:]
    p = getvar(ncfile, "p", units="hPa", timeidx=ALL_TIMES)[:,:ztop,:,:]
    
    ua30 = ua.resample(Time='30min').mean(dim='Time')
    ua30['Time'] = ua30['Time'] - utc2loc
    va30 = va.resample(Time='30min').mean(dim='Time')
    va30['Time'] = va30['Time'] - utc2loc
    wa30 = wa.resample(Time='30min').mean(dim='Time')
    wa30['Time'] = wa30['Time'] - utc2loc
    
    tv30 = tv.resample(Time='30min').mean(dim='Time')
    tv30['Time'] = tv30['Time'] - utc2loc
    z30 = z.resample(Time='30min').mean(dim='Time')
    z30['Time'] = z30['Time'] - utc2loc
    p30 = p.resample(Time='30min').mean(dim='Time')
    p30['Time'] = p30['Time'] - utc2loc

    ds1 = ua30.to_dataset(name='ua30')
    ds2 = va30.to_dataset(name='va30')
    ds3 = wa30.to_dataset(name='wa30')
    ds4 = tv30.to_dataset(name='tv30')
    ds5 = z30.to_dataset(name='z30')
    ds6 = p30.to_dataset(name='p30')
    dswrf = ds6.combine_first(ds5.combine_first(ds4.combine_first(ds3.combine_first(ds2.combine_first(ds1)))))
    dswrf.to_netcdf('interp30/'+str(expnames[k])+'.nc')
    print('done '+str(k)+' '+exprun)
    k=k+1

C:\Users\dell\Anaconda3\lib\site-packages\xarray\coding\times.py:240: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  dates = pd.to_datetime(dates, box=False)


done 0 20120816_d03_gfs05_p1r1


C:\Users\dell\Anaconda3\lib\site-packages\wrf\util.py:779: RuntimeWarning: invalid value encountered in equal
  lats = first_wrfnc.variables[lat_coord][:]
C:\Users\dell\Anaconda3\lib\site-packages\wrf\util.py:780: RuntimeWarning: invalid value encountered in equal
  lons = first_wrfnc.variables[lon_coord][:]
C:\Users\dell\Anaconda3\lib\site-packages\wrf\util.py:659: RuntimeWarning: invalid value encountered in equal
  lats = wrfnc.variables[latvar][:]
C:\Users\dell\Anaconda3\lib\site-packages\wrf\util.py:660: RuntimeWarning: invalid value encountered in equal
  lons = wrfnc.variables[lonvar][:]
C:\Users\dell\Anaconda3\lib\site-packages\wrf\util.py:1255: RuntimeWarning: invalid value encountered in equal
  data = var[time_idx_or_slice, :]
C:\Users\dell\Anaconda3\lib\site-packages\wrf\util.py:1310: RuntimeWarning: invalid value encountered in equal
  lon_coord_vals = lon_var[:]
C:\Users\dell\Anaconda3\lib\site-packages\wrf\util.py:1323: RuntimeWarning: invalid value encountered in equal


ValueError: index must be monotonic for resampling

In [ ]:
# (UV DIR, times, level, lat, lon) [0,:,:,20,56]
# SODAR   LAT 19.28, LON 103.77
k=1

for exprun in expruns:
    #exprun = '20120816_ds083_p1s1'
    f= '/mnt/sda2/exp6_llj/d03/'+exprun+'.nc'
    ncfile = Dataset(f)

    uv, wdir = getvar(ncfile, "uvmet_wspd_wdir", units="m s-1", timeidx=ALL_TIMES)
    tmpv = getvar(ncfile, "tv", units="degC", timeidx=ALL_TIMES)

    int_levs = [.030, .040, .050, .100, .150, .200, .250, .300, .350, .400, .450, .500]

    wind_ds = vinterp(ncfile, field=uv, vert_coord='ght_agl', interp_levels=int_levs, timeidx=ALL_TIMES, field_type='z')
    seriesw = wind_ds[:,:,[22,23],[18,19]].mean(dim=['south_north', 'west_east']).resample(Time='30min').mean(dim='Time')
    seriesw['Time'] = seriesw['Time'] - utc2loc
    seriesw.to_pandas().to_pickle('interp/wind_'+exprun+'.pk')

    wdir_ds = vinterp(ncfile, field=wdir, vert_coord='ght_agl', interp_levels=int_levs, timeidx=ALL_TIMES) 
    seriesd = wdir_ds[:,:,[22,23],[18,19]].mean(dim=['south_north', 'west_east']).resample(Time='30min').mean(dim='Time')
    seriesd['Time'] = seriesd['Time'] - utc2loc
    seriesd.to_pandas().to_pickle('interp/wdir_'+exprun+'.pk')

    tmpv_ds = vinterp(ncfile, field=tmpv, vert_coord='ght_agl', interp_levels=int_levs, timeidx=ALL_TIMES) 
    seriest = tmpv_ds[:,:,[22,23],[18,19]].mean(dim=['south_north', 'west_east']).resample(Time='30min').mean(dim='Time')
    seriest['Time'] = seriest['Time'] - utc2loc
    seriest.to_pandas().to_pickle('interp/tmpv_'+exprun+'.pk')
    print('done '+str(k)+' '+exprun)
    k=k+1


In [ ]:
 #LEV, LAT, LON      LAT 19.28,   LON 103.77  
f = '/home/wrf/WPS/met_em.d01.2012-08-17_00:00:00.nc'
ncfile = Dataset(f)
uv = getvar(ncfile, "uvmet_wspd_wdir", units="m s-1", timeidx=0)[0]
wind = uv[0,22,22]
print(uv.shape, wind.values, wind['XLAT_M'].values, wind['XLONG_M'].values)
print('')
Dataset.close
f = '/home/wrf/WPS/met_em.d02.2012-08-17_00:00:00.nc'
ncfile2 = Dataset(f)
uv2 = getvar(ncfile2, "uvmet_wspd_wdir", units="m s-1", timeidx=0)[0]
wind2 = uv2[0,22,20]
print(uv2.shape, wind2.values, wind2['XLAT_M'].values, wind2['XLONG_M'].values)
Dataset.close
print('')
f = '/home/wrf/WPS/met_em.d03.2012-08-17_00:00:00.nc'
ncfile3 = Dataset(f)
uv3 = getvar(ncfile3, "uvmet_wspd_wdir", units="m s-1", timeidx=0)[0]
wind3 = uv3[0,24,19]
print(uv3.shape, wind3.values, wind3['XLAT_M'].values, wind3['XLONG_M'].values)
Dataset.close

In [ ]:
uv[:15,20,:].plot.contourf()

In [ ]:
tv = getvar(ncfile, "tv")
tv[:15,20,:].plot.contourf()

In [ ]:
#wrf.extract_vars(wrfin, timeidx, varnames
p = getvar(ncfile, "p")
p[:15,20,:].plot.contourf()

In [ ]:
dd = pd.read_pickle('sodar_dd.pk')
dd = dd.interpolate()
sp = pd.read_pickle('sodar_sp.pk')
sp = sp.interpolate()
tv = pd.read_pickle('sodar_tv.pk')
tv = tv.interpolate()

In [ ]:
from wrf import to_np, getvar, CoordPair, vertcross
f = '/mnt/sda2/exp5_llj/201303_nam218.p1s1.nc'
ncfile = Dataset(exp+f)
p = getvar(ncfile, "pressure", timeidx=0)
lats = getvar(ncfile, "lat", timeidx=0)
lons = getvar(ncfile, "lon", timeidx=0)
print(lats.min(), lats.max())
print(lons.min(), lons.max())